### USE THIS BLOCK OF CODE - Hoover Index

In [19]:
from gurobipy import *
import math
import numpy as np

# Haversine function for distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

# set parameters
# Parameters
nodes, supply, demand, fixed_cost, lat, lon, pop, pci, pop_ratio = multidict({
                                ('node1'):[120,15,2241, 36.0175, 98.9245, 89806, 22407, .0003],
                             ('node2'):[210,518,2024, 36.6771, 95.9410, 307093, 20240, .00103],
                             ('node3'):[300,74,2163, 34.5203, 97.8722, 247970, 21626, .00083],
                             ('node4'):[150,586,2005, 35.7111, 95.3103,310785, 20051, .00104],
                             ('node5'):[90,252,1763, 34.9879, 95.8143,93245, 17632, .00031],
                             ('node6'):[180,238,2240, 35.2754, 97.0068, 452614, 22400, .00152],
                             ('node7'):[300,711,2677, 36.1593, 95.9410, 677358, 26769, .00227],
                             ('node8'):[480,559,2572, 35.6038, 97.3517, 802559, 25723, .00269]})


## Set arc, distance
arc, distance = multidict({('node2', 'node6'):[0],
                          ('node6', 'node8'):[0],
                          ('node2', 'node7'):[0],
                          ('node4', 'node7'):[0],
                          ('node3', 'node6'):[0],
                          ('node2', 'node4'):[0],
                          ('node4', 'node5'):[0],
                          ('node1', 'node3'):[0],
                          ('node1', 'node6'):[0],
                          ('node4', 'node6'):[0],
                          ('node5', 'node6'):[0],
                          ('node3', 'node5'):[0],
                          ('node1', 'node2'):[0],
                          ('node6', 'node2'):[0],
                          ('node8', 'node6'):[0],
                          ('node7', 'node2'):[0],
                          ('node7', 'node4'):[0],
                          ('node6', 'node3'):[0],
                          ('node4', 'node2'):[0],
                          ('node5', 'node4'):[0],
                          ('node3', 'node1'):[0],
                          ('node6', 'node1'):[0],
                          ('node6', 'node4'):[0],
                          ('node6', 'node5'):[0],
                          ('node5', 'node3'):[0],
                          ('node2', 'node1'):[0],
                          ('node1', 'node1'):[0],
                           ('node2', 'node2'):[0],
                           ('node3', 'node3'):[0],
                           ('node4', 'node4'):[0],
                           ('node5', 'node5'):[0],
                           ('node6', 'node6'):[0],
                           ('node7', 'node7'):[0],
                           ('node8', 'node8'):[0]})
# create model
m = Model("Final_Project")

# Variables
# Set variables
x = {}
y = {}
          
# Set objective
x = m.addVars(arc,vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY, name = 'x')
y = m.addVars(nodes,vtype=GRB.INTEGER, lb=0, ub =GRB.INFINITY, name = 'y')

P = math.ceil((sum(demand[i] for i in nodes) - sum(supply[i] for i in nodes))/30)

# Make sure the number of facilities can hold the supply or have extr beds
m.addConstrs((x.sum(i, '*') - x.sum('*', i) >= supply[i] - demand[i] for i in nodes), "Meet or Exceed Demand")

# make sure the number of facilities located doesnt exceed max facilities allowed
m.addConstr((quicksum(y[j] for j in nodes) <= P), 'facilities met')

### FAIRNESS MEASURES
n = len(nodes)
avg_facilities = quicksum(y[j] for j in nodes) / n

# Create auxiliary variables
u = m.addVars(nodes, vtype=GRB.CONTINUOUS, name='u')

# Linearize constraints for absolute value
m.addConstrs((u[i] >= y[i] - avg_facilities for i in nodes), name='abs_lower')
m.addConstrs((u[i] >= avg_facilities - y[i] for i in nodes ), name='abs_upper')

# Calculate Hoover index
hoover_sum = quicksum(u[i] for i in nodes)
m.addConstr(hoover_sum <= 0.3 * n * avg_facilities, name='hoover_index')


# Set Objective and Priorities    
z1 = quicksum(30*fixed_cost[j]*y[j] for j in nodes)
z3 = quicksum((haversine(lat[i], lon[i], lat[j], lon[j])**2) * x[i, j] for (i, j) in arc)
z2 = quicksum(y[j] for j in nodes)
m.update()
m.NumObj = 3
m.setObjectiveN(z1, GRB.MINIMIZE, priority=1)
m.setObjectiveN(z3, GRB.MINIMIZE, priority=2)
m.setObjectiveN(-z2, GRB.MINIMIZE, priority=0)


m.optimize()


# print outputs
if m.status == GRB.OPTIMAL:
    print('**Optimal Solution Found**\n')
    print('--Objective Function--\n %g\n' % m.objVal)
    
#     print('--Decision Variables--')
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
m.printAttr('X')

print(P)
print(sum(supply[i] for i in nodes))
print(sum(demand[i] for i in nodes))
print(z1)

m.optimize()
if m.status == GRB.OPTIMAL:
    print('Facilities:')
    for i in nodes:
        print(i, round(y[i].x,0), round(30*fixed_cost[i]*y[i].x,0))
    print(sum(round(y[i].x,0) for i in nodes))    
    print(sum(round(30 * fixed_cost[i] * y[i].x, 0) for i in nodes))



SyntaxError: invalid decimal literal (3274468798.py, line 98)

### Gini Index

In [132]:
from gurobipy import *
import math
import numpy as np

# Haversine function for distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

# set parameters
# Parameters
nodes, supply, demand, fixed_cost, lat, lon, pop, pci, pop_ratio = multidict({
                                ('node1'):[120,15,2241, 36.0175, 98.9245, 89806, 22407, .0003],
                             ('node2'):[210,518,2024, 36.6771, 95.9410, 307093, 20240, .00103],
                             ('node3'):[300,74,2163, 34.5203, 97.8722, 247970, 21626, .00083],
                             ('node4'):[150,586,2005, 35.7111, 95.3103,310785, 20051, .00104],
                             ('node5'):[90,252,1763, 34.9879, 95.8143,93245, 17632, .00031],
                             ('node6'):[180,238,2240, 35.2754, 97.0068, 452614, 22400, .00152],
                             ('node7'):[300,711,2677, 36.1593, 95.9410, 677358, 26769, .00227],
                             ('node8'):[480,559,2572, 35.6038, 97.3517, 802559, 25723, .00269]})


## Set arc, distance
arc, distance = multidict({('node2', 'node6'):[0],
                          ('node6', 'node8'):[0],
                          ('node2', 'node7'):[0],
                          ('node4', 'node7'):[0],
                          ('node3', 'node6'):[0],
                          ('node2', 'node4'):[0],
                          ('node4', 'node5'):[0],
                          ('node1', 'node3'):[0],
                          ('node1', 'node6'):[0],
                          ('node4', 'node6'):[0],
                          ('node5', 'node6'):[0],
                          ('node3', 'node5'):[0],
                          ('node1', 'node2'):[0],
                          ('node6', 'node2'):[0],
                          ('node8', 'node6'):[0],
                          ('node7', 'node2'):[0],
                          ('node7', 'node4'):[0],
                          ('node6', 'node3'):[0],
                          ('node4', 'node2'):[0],
                          ('node5', 'node4'):[0],
                          ('node3', 'node1'):[0],
                          ('node6', 'node1'):[0],
                          ('node6', 'node4'):[0],
                          ('node6', 'node5'):[0],
                          ('node5', 'node3'):[0],
                          ('node2', 'node1'):[0],
                          ('node1', 'node1'):[0],
                           ('node2', 'node2'):[0],
                           ('node3', 'node3'):[0],
                           ('node4', 'node4'):[0],
                           ('node5', 'node5'):[0],
                           ('node6', 'node6'):[0],
                           ('node7', 'node7'):[0],
                           ('node8', 'node8'):[0]})
# create model
m = Model("Final_Project")

# Variables
# Set variables
x = {}
y = {}


          
# Set objective
x = m.addVars(arc,vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY, name = 'x')
y = m.addVars(nodes,vtype=GRB.INTEGER, lb=0, ub =GRB.INFINITY, name = 'y')
 



P = math.ceil((sum(demand[i] for i in nodes) - sum(supply[i] for i in nodes))/30)

# Make sure the number of facilities can hold the supply or have extr beds
m.addConstrs((x.sum(i, '*') - x.sum('*', i) >= supply[i] - demand[i] for i in nodes), "Meet or Exceed Demand")

# make sure the number of facilities located doesnt exceed max facilities allowed
m.addConstr((quicksum(y[j] for j in nodes) <= P), 'facilities met')





### FAIRNESS MEASURES
# # all for an even distribution of facilities in each node
# m.addVars(nodes, vtype=GRB.CONTINUOUS, name='pci_ratio')

# get pop_tot
budget_state = 527000000

# Create variables for pci_ratio


#m.addConstrs((quicksum(30*y[i]*fixed_cost[i]/pop_ratio[i]*budget_state for i in nodes) >= 0.100) for nodes in arc)
#m.addConstrs((quicksum(30*y[i]*fixed_cost[i]/pop[i]*pci[i] for i in nodes) >= 0.25) for nodes in arc)

n = len(nodes)
avg_facilities = quicksum(y[j] for j in nodes) / n

# Create auxiliary variables
u = m.addVars(nodes, vtype=GRB.CONTINUOUS, name='u')

# Linearize constraints for absolute value
m.addConstrs((u[i] >= y[i] - y[j] for i in nodes for j in nodes if i != j), name='gini_lower')
m.addConstrs((u[i] >= y[j] - y[i] for i in nodes for j in nodes if i != j), name='gini_upper')

# Calculate Hoover index
gini_sum = quicksum(u[i] for i in nodes)
m.addConstr(gini_sum <= 0.3* n **2* avg_facilities, name='gini_index')







# Set Objective and Priorities    
z1 = quicksum(30*fixed_cost[j]*y[j] for j in nodes)
z3 = quicksum((haversine(lat[i], lon[i], lat[j], lon[j])**2) * x[i, j] for (i, j) in arc)
z2 = quicksum(y[j] for j in nodes)
m.update()
m.NumObj = 3
m.setObjectiveN(z1, GRB.MINIMIZE, priority=1)
m.setObjectiveN(z3, GRB.MINIMIZE, priority=2)
m.setObjectiveN(-z2, GRB.MINIMIZE, priority=0)


m.optimize()


# print outputs
if m.status == GRB.OPTIMAL:
    print('**Optimal Solution Found**\n')
    print('--Objective Function--\n %g\n' % m.objVal)
    
#     print('--Decision Variables--')
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
m.printAttr('X')

print(P)
print(sum(supply[i] for i in nodes))
print(sum(demand[i] for i in nodes))
print(z1)

m.optimize()
if m.status == GRB.OPTIMAL:
    print('Facilities:')
    for i in nodes:
        print(i, round(y[i].x,0), round(30*fixed_cost[i]*y[i].x,0))
    print(sum(round(y[i].x,0) for i in nodes))    
    print(sum(round(30 * fixed_cost[i] * y[i].x, 0) for i in nodes))



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 42 columns and 60 nonzeros
Model fingerprint: 0x38087ff6
Variable types: 0 continuous, 42 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 9 rows, 

### Hoover - modified

In [12]:
from gurobipy import *
import math
import numpy as np

# Haversine function for distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

# set parameters
# Parameters
nodes, supply, demand, fixed_cost, lat, lon, pop, pci, pop_ratio = multidict({
                                ('node1'):[120,15,2241, 36.0175, 98.9245, 89806, 22407, .0003],
                             ('node2'):[210,518,2024, 36.6771, 95.9410, 307093, 20240, .00103],
                             ('node3'):[300,74,2163, 34.5203, 97.8722, 247970, 21626, .00083],
                             ('node4'):[150,586,2005, 35.7111, 95.3103,310785, 20051, .00104],
                             ('node5'):[90,252,1763, 34.9879, 95.8143,93245, 17632, .00031],
                             ('node6'):[180,238,2240, 35.2754, 97.0068, 452614, 22400, .00152],
                             ('node7'):[300,711,2677, 36.1593, 95.9410, 677358, 26769, .00227],
                             ('node8'):[480,559,2572, 35.6038, 97.3517, 802559, 25723, .00269]})


## Set arc, distance
arc, distance = multidict({('node2', 'node6'):[0],
                          ('node6', 'node8'):[0],
                          ('node2', 'node7'):[0],
                          ('node4', 'node7'):[0],
                          ('node3', 'node6'):[0],
                          ('node2', 'node4'):[0],
                          ('node4', 'node5'):[0],
                          ('node1', 'node3'):[0],
                          ('node1', 'node6'):[0],
                          ('node4', 'node6'):[0],
                          ('node5', 'node6'):[0],
                          ('node3', 'node5'):[0],
                          ('node1', 'node2'):[0],
                          ('node6', 'node2'):[0],
                          ('node8', 'node6'):[0],
                          ('node7', 'node2'):[0],
                          ('node7', 'node4'):[0],
                          ('node6', 'node3'):[0],
                          ('node4', 'node2'):[0],
                          ('node5', 'node4'):[0],
                          ('node3', 'node1'):[0],
                          ('node6', 'node1'):[0],
                          ('node6', 'node4'):[0],
                          ('node6', 'node5'):[0],
                          ('node5', 'node3'):[0],
                          ('node2', 'node1'):[0],
                          ('node1', 'node1'):[0],
                           ('node2', 'node2'):[0],
                           ('node3', 'node3'):[0],
                           ('node4', 'node4'):[0],
                           ('node5', 'node5'):[0],
                           ('node6', 'node6'):[0],
                           ('node7', 'node7'):[0],
                           ('node8', 'node8'):[0]})
# create model
m = Model("Final_Project")

# Variables
# Set variables
x = {}
y = {}
          
# Set objective
x = m.addVars(arc,vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY, name = 'x')
y = m.addVars(nodes,vtype=GRB.INTEGER, lb=0, ub =GRB.INFINITY, name = 'y')

P = math.ceil((sum(demand[i] for i in nodes) - sum(supply[i] for i in nodes))/30)

# Make sure the number of facilities can hold the supply or have extr beds
m.addConstrs((x.sum(i, '*') - x.sum('*', i) >= supply[i] - demand[i] for i in nodes), "Meet or Exceed Demand")

# make sure the number of facilities located doesnt exceed max facilities allowed
m.addConstr((quicksum(y[j] for j in nodes) <= P), 'facilities met')

### FAIRNESS MEASURES
n = len(nodes)
avg_facilities = quicksum(y[j] for j in nodes) / n

# Create auxiliary variables
u = m.addVars(nodes, vtype=GRB.CONTINUOUS, name='u')

# Linearize constraints for absolute value
m.addConstrs((u[i] >= y[i] - avg_facilities for i in nodes), name='abs_lower')
m.addConstrs((u[i] >= avg_facilities - y[i] for i in nodes), name='abs_upper')


# Calculate Hoover index
# Calculate Hoover index
hoover_sum = quicksum(u[i] for i in nodes)
m.addConstr(hoover_sum <= 0.1 * 2*n * avg_facilities, name='hoover_index')


# Set Objective and Priorities    
z1 = quicksum(30*fixed_cost[j]*y[j] for j in nodes)
z3 = quicksum((haversine(lat[i], lon[i], lat[j], lon[j])**2) * x[i, j] for (i, j) in arc)
z2 = quicksum(y[j] for j in nodes)
m.update()
m.NumObj = 3
m.setObjectiveN(z1, GRB.MINIMIZE, priority=1)
m.setObjectiveN(z3, GRB.MINIMIZE, priority=2)
m.setObjectiveN(-z2, GRB.MINIMIZE, priority=0)

#m.setParam('NonConvex', 2)
m.optimize()


# print outputs
if m.status == GRB.OPTIMAL:
    print('**Optimal Solution Found**\n')
    print('--Objective Function--\n %g\n' % m.objVal)
    
#     print('--Decision Variables--')
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
m.printAttr('X')

print(P)
print(sum(supply[i] for i in nodes))
print(sum(demand[i] for i in nodes))
print(z1)

m.optimize()
if m.status == GRB.OPTIMAL:
    print('Facilities:')
    for i in nodes:
        print(i, round(y[i].x,0), round(30*fixed_cost[i]*y[i].x,0))
    print(sum(round(y[i].x,0) for i in nodes))    
    print(sum(round(30 * fixed_cost[i] * y[i].x, 0) for i in nodes))



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26 rows, 50 columns and 220 nonzeros
Model fingerprint: 0xb36a38a2
Variable types: 8 continuous, 42 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 26 row

### Gini - modified

In [15]:
from gurobipy import *
import math
import numpy as np

# Haversine function for distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

# set parameters
# Parameters
nodes, supply, demand, fixed_cost, lat, lon, pop, pci, pop_ratio = multidict({
                                ('node1'):[120,15,2241, 36.0175, 98.9245, 89806, 22407, .0003],
                             ('node2'):[210,518,2024, 36.6771, 95.9410, 307093, 20240, .00103],
                             ('node3'):[300,74,2163, 34.5203, 97.8722, 247970, 21626, .00083],
                             ('node4'):[150,586,2005, 35.7111, 95.3103,310785, 20051, .00104],
                             ('node5'):[90,252,1763, 34.9879, 95.8143,93245, 17632, .00031],
                             ('node6'):[180,238,2240, 35.2754, 97.0068, 452614, 22400, .00152],
                             ('node7'):[300,711,2677, 36.1593, 95.9410, 677358, 26769, .00227],
                             ('node8'):[480,559,2572, 35.6038, 97.3517, 802559, 25723, .00269]})


## Set arc, distance
arc, distance = multidict({('node2', 'node6'):[0],
                          ('node6', 'node8'):[0],
                          ('node2', 'node7'):[0],
                          ('node4', 'node7'):[0],
                          ('node3', 'node6'):[0],
                          ('node2', 'node4'):[0],
                          ('node4', 'node5'):[0],
                          ('node1', 'node3'):[0],
                          ('node1', 'node6'):[0],
                          ('node4', 'node6'):[0],
                          ('node5', 'node6'):[0],
                          ('node3', 'node5'):[0],
                          ('node1', 'node2'):[0],
                          ('node6', 'node2'):[0],
                          ('node8', 'node6'):[0],
                          ('node7', 'node2'):[0],
                          ('node7', 'node4'):[0],
                          ('node6', 'node3'):[0],
                          ('node4', 'node2'):[0],
                          ('node5', 'node4'):[0],
                          ('node3', 'node1'):[0],
                          ('node6', 'node1'):[0],
                          ('node6', 'node4'):[0],
                          ('node6', 'node5'):[0],
                          ('node5', 'node3'):[0],
                          ('node2', 'node1'):[0],
                          ('node1', 'node1'):[0],
                           ('node2', 'node2'):[0],
                           ('node3', 'node3'):[0],
                           ('node4', 'node4'):[0],
                           ('node5', 'node5'):[0],
                           ('node6', 'node6'):[0],
                           ('node7', 'node7'):[0],
                           ('node8', 'node8'):[0]})
# create model
m = Model("Final_Project")

# Variables
# Set variables
x = {}
y = {}


          
# Set objective
x = m.addVars(arc,vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY, name = 'x')
y = m.addVars(nodes,vtype=GRB.INTEGER, lb=0, ub =GRB.INFINITY, name = 'y')
 



P = math.ceil((sum(demand[i] for i in nodes) - sum(supply[i] for i in nodes))/30)

# Make sure the number of facilities can hold the supply or have extr beds
m.addConstrs((x.sum(i, '*') - x.sum('*', i) >= supply[i] - demand[i] for i in nodes), "Meet or Exceed Demand")

# make sure the number of facilities located doesnt exceed max facilities allowed
m.addConstr((quicksum(y[j] for j in nodes) <= P), 'facilities met')





### FAIRNESS MEASURES
# # all for an even distribution of facilities in each node
# m.addVars(nodes, vtype=GRB.CONTINUOUS, name='pci_ratio')

# get pop_tot
budget_state = 527000000

# Create variables for pci_ratio


#m.addConstrs((quicksum(30*y[i]*fixed_cost[i]/pop_ratio[i]*budget_state for i in nodes) >= 0.100) for nodes in arc)
#m.addConstrs((quicksum(30*y[i]*fixed_cost[i]/pop[i]*pci[i] for i in nodes) >= 0.25) for nodes in arc)

n = len(nodes)
avg_facilities = quicksum(y[j] for j in nodes) / n

# Create auxiliary variables
u = m.addVars(nodes, vtype=GRB.CONTINUOUS, name='u')

# Linearize constraints for absolute value
m.addConstrs((u[i] >= y[i] - y[j] for i in nodes for j in nodes if i != j), name='gini_lower')
m.addConstrs((u[i] >= y[j] - y[i] for i in nodes for j in nodes if i != j), name='gini_upper')

# Calculate Hoover index
gini_sum = quicksum(u[i] for i in nodes)
m.addConstr(gini_sum <= 0.3*2* n **2* avg_facilities, name='gini_index')

# # Create binary variables
# z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name='z')

# # Linearize the product term
# m.addConstrs((gini_sum >= 0.5 * n * (n - 1) * avg_facilities - (n - 1) * z[i, j]) for i in nodes for j in nodes if i != j)
# m.addConstrs((gini_sum <= 0.5 * n * (n - 1) * avg_facilities + (n - 1) * z[i, j]) for i in nodes for j in nodes if i != j)

# # Add linearization constraint for Gini index
# m.addConstr(0.5 <= 0.5 * n * (n - 1) * avg_facilities - quicksum((n - 1) * z[i, j] for i in nodes for j in nodes if i != j), name='gini_index')

# m.addConstr(0.5 <= 0.5 * n - 1) * avg_facilities - quicksum((n - 1) * z[i, j] for i in nodes for j in nodes if i != j), name='gini_index')








# Set Objective and Priorities    
z1 = quicksum(30*fixed_cost[j]*y[j] for j in nodes)
z3 = quicksum((haversine(lat[i], lon[i], lat[j], lon[j])**2) * x[i, j] for (i, j) in arc)
z2 = quicksum(y[j] for j in nodes)
m.update()
m.NumObj = 3
m.setObjectiveN(z1, GRB.MINIMIZE, priority=1)
m.setObjectiveN(z3, GRB.MINIMIZE, priority=2)
m.setObjectiveN(-z2, GRB.MINIMIZE, priority=0)

#m.setParam('NonConvex', 2)
#m.setParam('IterationLimit', 3000)
#m.setParam('FlowCoverCuts', 2)

m.optimize()
m.printQuality()

# print outputs
if m.status == GRB.OPTIMAL:
    print('**Optimal Solution Found**\n')
    print('--Objective Function--\n %g\n' % m.objVal)
    
#     print('--Decision Variables--')
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
m.printAttr('X')

print(P)
print(sum(supply[i] for i in nodes))
print(sum(demand[i] for i in nodes))
print(z1)

m.optimize()
if m.status == GRB.OPTIMAL:
    print('Facilities:')
    for i in nodes:
        print(i, round(y[i].x,0), round(30*fixed_cost[i]*y[i].x,0))
    print(sum(round(y[i].x,0) for i in nodes))    
    print(sum(round(30 * fixed_cost[i] * y[i].x, 0) for i in nodes))



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 122 rows, 50 columns and 412 nonzeros
Model fingerprint: 0x896a15ab
Variable types: 8 continuous, 42 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 122 r

### QoE Index

In [37]:
from gurobipy import *
import math
import numpy as np
import statistics

# Haversine function for distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d
nodes = ['node1', 'node2', 'node3', 'node4', 'node5', 'node6', 'node7', 'node8']
# set parameters
# Parameters
nodes, supply, demand, fixed_cost, lat, lon, pop, pci, pop_ratio = multidict({
                                ('node1'):[120,15,2241, 36.0175, 98.9245, 89806, 22407, .0003],
                             ('node2'):[210,518,2024, 36.6771, 95.9410, 307093, 20240, .00103],
                             ('node3'):[300,74,2163, 34.5203, 97.8722, 247970, 21626, .00083],
                             ('node4'):[150,586,2005, 35.7111, 95.3103,310785, 20051, .00104],
                             ('node5'):[90,252,1763, 34.9879, 95.8143,93245, 17632, .00031],
                             ('node6'):[180,238,2240, 35.2754, 97.0068, 452614, 22400, .00152],
                             ('node7'):[300,711,2677, 36.1593, 95.9410, 677358, 26769, .00227],
                             ('node8'):[480,559,2572, 35.6038, 97.3517, 802559, 25723, .00269]})


## Set arc, distance
arc, distance = multidict({('node2', 'node6'):[0],
                          ('node6', 'node8'):[0],
                          ('node2', 'node7'):[0],
                          ('node4', 'node7'):[0],
                          ('node3', 'node6'):[0],
                          ('node2', 'node4'):[0],
                          ('node4', 'node5'):[0],
                          ('node1', 'node3'):[0],
                          ('node1', 'node6'):[0],
                          ('node4', 'node6'):[0],
                          ('node5', 'node6'):[0],
                          ('node3', 'node5'):[0],
                          ('node1', 'node2'):[0],
                          ('node6', 'node2'):[0],
                          ('node8', 'node6'):[0],
                          ('node7', 'node2'):[0],
                          ('node7', 'node4'):[0],
                          ('node6', 'node3'):[0],
                          ('node4', 'node2'):[0],
                          ('node5', 'node4'):[0],
                          ('node3', 'node1'):[0],
                          ('node6', 'node1'):[0],
                          ('node6', 'node4'):[0],
                          ('node6', 'node5'):[0],
                          ('node5', 'node3'):[0],
                          ('node2', 'node1'):[0],
                          ('node1', 'node1'):[0],
                           ('node2', 'node2'):[0],
                           ('node3', 'node3'):[0],
                           ('node4', 'node4'):[0],
                           ('node5', 'node5'):[0],
                           ('node6', 'node6'):[0],
                           ('node7', 'node7'):[0],
                           ('node8', 'node8'):[0]})
# create model
m = Model("Final_Project")

# Variables
# Set variables
x = {}
y = {}


          
# Set objective
x = m.addVars(arc,vtype=GRB.INTEGER, lb = 0, ub = GRB.INFINITY, name = 'x')
y = m.addVars(nodes,vtype=GRB.INTEGER, lb=0, ub =GRB.INFINITY, name = 'y')
 



P = math.ceil((sum(demand[i] for i in nodes) - sum(supply[i] for i in nodes))/30)

# Make sure the number of facilities can hold the supply or have extr beds
m.addConstrs((x.sum(i, '*') - x.sum('*', i) >= supply[i] - demand[i] for i in nodes), "Meet or Exceed Demand")

# make sure the number of facilities located doesnt exceed max facilities allowed
m.addConstr((quicksum(y[j] for j in nodes) <= P), 'facilities met')





### FAIRNESS MEASURES
# # all for an even distribution of facilities in each node
# m.addVars(nodes, vtype=GRB.CONTINUOUS, name='pci_ratio')

# get pop_tot
budget_state = 527000000
n = len(nodes)



# Calculate the standard deviation, highest, and lowest QoE values
sigma = statistics.stdev(qoe_values)
highest_qoe =38
lowest_qoe = 0

# Calculate the fairness metric
fairness_metric = 1 - (2 * sigma / (highest_qoe - lowest_qoe))

# Set a desired threshold for fairness (e.g., minimum fairness metric required)
desired_fairness = 0.6

# Add the fairness constraint to the model
m.addConstr(fairness_metric >= desired_fairness, "fairness_constraint")

# Set Objective and Priorities    
z1 = quicksum(30*fixed_cost[j]*y[j] for j in nodes)
z3 = quicksum((haversine(lat[i], lon[i], lat[j], lon[j])**2) * x[i, j] for (i, j) in arc)
z2 = quicksum(y[j] for j in nodes)
m.update()
m.NumObj = 3
m.setObjectiveN(z1, GRB.MINIMIZE, priority=1)
m.setObjectiveN(z3, GRB.MINIMIZE, priority=2)
m.setObjectiveN(-z2, GRB.MINIMIZE, priority=0)

#m.setParam('NonConvex', 2)
m.optimize()


# print outputs
if m.status == GRB.OPTIMAL:
    print('**Optimal Solution Found**\n')
    print('--Objective Function--\n %g\n' % m.objVal)
    
#     print('--Decision Variables--')
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
m.printAttr('X')

print(P)
print(sum(supply[i] for i in nodes))
print(sum(demand[i] for i in nodes))
print(z1)

m.optimize()
if m.status == GRB.OPTIMAL:
    print('Facilities:')
    for i in nodes:
        print(i, round(y[i].x,0), round(30*fixed_cost[i]*y[i].x,0))
    print(sum(round(y[i].x,0) for i in nodes))    
    print(sum(round(30 * fixed_cost[i] * y[i].x, 0) for i in nodes))



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 42 columns and 60 nonzeros
Model fingerprint: 0x456a0d25
Variable types: 0 continuous, 42 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 10 rows

In [20]:
print("Decision Variables:")
for node in nodes:
    var = m.getVarByName(node)
    print(f"{var.varName}: {var.x}")


Decision Variables:


AttributeError: 'NoneType' object has no attribute 'varName'